# Parameter estimation for a linear operator with pyGPs

Assumptions: <br>
$\mathcal{L}_x^{\phi}u(x) = f(x)$ <br>
$u(x) \sim \mathcal{GP}(0, k_{uu}(x, x', \theta))$ <br>
$f(x) \sim \mathcal{GP}(0, k_{ff}(x, x', \theta, \phi))$ <br>
$\theta = \{\sigma, l\}$ <br>
<br>
Chosen operator:
$\mathcal{L}_x^{\phi}u(x) = \phi*u(x)$ <br> <br>
We choose the following two functions to sample the data: <br>
$u(x) = \sqrt{x}$ <br>
$f(x) = 12*u(x) = 12 \sqrt{x}$ <br>
<br>
Problem at hand: Given $\{X_u, y_u\}$ and $\{X_f, y_f\}$, estimate $\phi$. <br>
<br>
We employ a GP with a RBF kernel for u and f: <br>
$k_{uu}(x_i, x_j; \theta_u) = \sigma_u^2 \exp(-\frac{1}{2l_u^2}(x_i - x_j)^2)$ <br>
$k_{ff}(x_i, x_j; \theta_f) = \sigma_f^2 \exp(-\frac{1}{2l_f^2}(x_i - x_j)^2)$ <br> 

We use the known transformation behavior of Gaussian Processes: <br>
$k_{ff}(x_i, x_j; \theta, \phi) \\
= \mathcal{L}_{x_i}^{\phi}\mathcal{L}_{x_j}^{\phi}k_{uu}(x_i, x_j; \theta) \\
= \phi^2\sigma_u^2 \exp(-\frac{1}{2l_u^2}(x_i - x_j)^2)$ <br> <br>

Equating the two expressions we have for $k_{ff}$ and comparing a diagonal entry (where $x_i = x_j$), it follows that
$\sigma_f^2 = \phi^2\sigma_u^2$, i.e. 
$\phi = \frac{\sigma_f}{\sigma_u}$.




In [ ]:
import numpy as np
import pyGPs

# Linear functional L = phi*u(x) was chosen. Desired result: phi = 12.0.

# Generating data
x_u = np.linspace(0, 2*np.pi, 15)
y_u = np.sqrt(x_u)               # Keeping it as simple as possible, with sin instead of sqrt the optimizer can't 
                                 # calculate the optimal hyperparameters, independent of the method
x_f = x_u
y_f = 12.0*np.sqrt(x_f)          # You can vary the factor, and that very factor should be the output of this program

# The function u is assumed to be a Gaussian Process. 
# After a linear transformation, f has to be a Gaussian Process as well.

model_u = pyGPs.GPR()
model_u.setData(x_u, y_u)
model_u.optimize(x_u, y_u)

model_f = pyGPs.GPR()
model_f.setData(x_f, y_f)
model_f.optimize(x_f, y_f)

# Note that in hyp only the logarithm of the hyperparameter is stored!
# Characteristic length-scale l is equal to np.exp(hyp[0]) (Default: 1)
# Signal variance sigma is equal to np.exp(hyp[1]) (Default: 1)

print(np.exp(model_f.covfunc.hyp[1])/np.exp(model_u.covfunc.hyp[1]))          # This should give 12 as output

# My Output: 12.0486915165